In [ ]:
# from FR3Env import getDataPath
import matplotlib.pyplot as plt
import numpy as np
# from pinocchio.robot_wrapper import RobotWrapper
from scipy.spatial.transform import Rotation as R
import pickle
import sys
import time
# sys.path.append("/usr/local/lib/python3.8/site-packages")
from dgh_franka.heads import FrankaDGH
import yaml

In [ ]:
franka_dgh = FrankaDGH(interface_type='joint_dyn_interface')

In [ ]:
nominal_joint_angles = np.array(
    [
        0.0,
        -np.pi / 4,
        0.0,
        -3 * np.pi / 4,
        0.0,
        np.pi / 2,
        np.pi / 4,
        0.001,
        0.001,
    ]
)

In [ ]:
start_time = time.time()
computation_times = []

for i in range(1000000):
    # read states and get time
    state = franka_dgh.readStates()
    q, dq = state["q"], state["dq"]
    q = q.copy()
    dq = dq.copy()
    
    tau = 10 * (nominal_joint_angles - q) + 1 * (0 - dq)

    # set control, last 2 terms are for the finger
    franka_dgh.setCommand(tau[0:7])
    computation_times.append(time.time() - start_time)
    time.sleep(0.001)

In [ ]:
franka_dgh.close()

In [ ]:
import os
pid = os.getpid()
print(f'The Process ID of the current notebook is: {pid}. Now setting it to real-time mode...')
!chrt -f -p 98 $pid # 99 is the process ID of the DGM. This notebook is lower in importan
!chrt -p $pid